In [2]:
from dash import Dash, html, dcc, Output, Input
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc
from pathlib import Path 
from datetime import datetime
import plotly.figure_factory as ff


In [3]:
df_defender = pd.read_csv('../../data/Stats_csv/Processed_Defender_data.csv')
df_forward = pd.read_csv('../../data/Stats_csv/Processed_Forward_data.csv')
df_goalkeeper = pd.read_csv('../../data/Stats_csv/Processed_Goalkeeper_data.csv')
df_midfielder = pd.read_csv('../../data/Stats_csv/Processed_Midfielder_data.csv')

Finding Distribution and the Average of Age of Players

In [4]:
today = datetime.now() 

df_defender['Date of Birth'] = pd.to_datetime(df_defender['Date of Birth'], dayfirst=True)
df_defender['Age'] = today.year - df_defender['Date of Birth'].dt.year

df_forward['Date of Birth'] = pd.to_datetime(df_forward['Date of Birth'], dayfirst=True)
df_forward['Age'] = today.year - df_forward['Date of Birth'].dt.year

df_goalkeeper['Date of Birth'] = pd.to_datetime(df_forward['Date of Birth'], dayfirst=True)
df_goalkeeper['Age'] = today.year - df_goalkeeper['Date of Birth'].dt.year

df_midfielder['Date of Birth'] = pd.to_datetime(df_forward['Date of Birth'], dayfirst=True)
df_midfielder['Age'] = today.year - df_midfielder['Date of Birth'].dt.year

hist_data_defender = [df_defender['Age'].tolist()]
hist_data_forward = [df_forward['Age'].tolist()]
hist_data_goalkeeper = [df_goalkeeper['Age'].tolist()]
hist_data_midfielder = [df_midfielder['Age'].tolist()]

colors = ['rgb(3, 87, 175)']
defender_group = ["Defender's Age"]
fig_defender = ff.create_distplot(hist_data_defender, defender_group,bin_size=1,colors=colors, show_rug=False)
fig_defender.update_layout(width=900, height=600, bargap=0.01)
fig_defender.show() 

colors = ['rgb(3, 87, 175)']
forward_group = ["Forward's Age"]
fig_forward = ff.create_distplot(hist_data_forward, forward_group,bin_size=1,colors=colors, show_rug=False)
fig_forward.update_layout(width=900, height=600, bargap=0.01)
fig_forward.show()

colors = ['rgb(3, 87, 175)']
goalkeeper_group = ["Goalkeeper's Age"]
fig_goalkeeper = ff.create_distplot(hist_data_goalkeeper, goalkeeper_group,bin_size=1,colors=colors, show_rug=False)
fig_goalkeeper.update_layout(width=900, height=600, bargap=0.01)
fig_goalkeeper.show()

# colors = ['rgb(3, 87, 175)']
# midfielder_group = ["Midfielder's Age"]
# fig_midfielder = ff.create_distplot(hist_data_midfielder, midfielder_group,bin_size=1,colors=colors, show_rug=False)
# fig_midfielder.update_layout(width=900, height=600, bargap=0.01)
# fig_midfielder.show()
print(hist_data_midfielder) #sao thông số lại có NaN trong khi hist_data khác bình thường

[[20.0, 27.0, 20.0, 28.0, 25.0, 22.0, 22.0, 22.0, 24.0, 24.0, 23.0, 25.0, 17.0, 31.0, 23.0, 24.0, 23.0, 20.0, 29.0, 27.0, 33.0, 27.0, 25.0, 27.0, 23.0, 19.0, 22.0, 32.0, 34.0, 25.0, 19.0, 28.0, 27.0, 27.0, 25.0, 25.0, 27.0, 19.0, 18.0, 24.0, 18.0, 20.0, 25.0, 18.0, 27.0, 19.0, 27.0, 23.0, 25.0, 22.0, 28.0, 19.0, 28.0, 24.0, 18.0, 24.0, 17.0, 22.0, 24.0, 37.0, 28.0, 27.0, 21.0, 28.0, 33.0, 23.0, 25.0, 25.0, 23.0, 25.0, 22.0, 24.0, 18.0, 23.0, 20.0, 30.0, 21.0, 16.0, 27.0, 18.0, 25.0, 18.0, 27.0, 21.0, 18.0, 25.0, 20.0, 28.0, 34.0, 32.0, 22.0, 26.0, 21.0, 31.0, 22.0, 26.0, 29.0, 21.0, 19.0, 26.0, 30.0, 24.0, 30.0, 25.0, 21.0, 33.0, 25.0, 27.0, 21.0, 23.0, 28.0, 27.0, 20.0, 17.0, 22.0, 32.0, 26.0, 27.0, 23.0, 28.0, 18.0, 30.0, 19.0, 21.0, 20.0, 20.0, 20.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n

Chỉ số của từng cầu thủ. Defender

In [12]:
selected_columns = ['Passes per match','Recoveries','Duels won','Tackles','Interceptions','Clearances']
df_subset1 = df_defender[df_defender['Name'] == 'Taylor Harwood-Bellis'] #Chọn cầu thủ trực tiếp

df_subset2 = df_defender[selected_columns]

r_values = df_subset2.iloc[0]

df_radar = pd.DataFrame(dict(
    r=r_values.values,        # Use the aggregated or raw values
    theta=r_values.index      # Use the column names as theta
))

theta = ['Passes per match','Recoveries','Duels won','Tackles','Interceptions','Clearances']
fig = px.line_polar(df_radar, r='r', theta='theta', line_close=True)
fig.update_traces(fill='toself')
fig.show()

Passes per match    68.54
Recoveries          40.00
Duels won           30.00
Tackles              7.00
Interceptions        8.00
Clearances          48.00
Name: 0, dtype: float64


Players over 25 and under  25 years old by club

In [ ]:
df1 = df_defender[df_defender['Age'] > 25] # dataset for players over 25
df1 = df_forward[df_defender['Age'] > 25] # dataset for players over 25
df1 = df_midfielder[df_defender['Age'] > 25] # dataset for players over 25
df2 = df_defender[df_forward['Age'] <= 25] # dataset for players under 25
df2 = df_forward[df_forward['Age'] <= 25] # dataset for players under 25
df2 = df_midfielder[df_forward['Age'] <= 25] # dataset for players under 25

plot_df = data[data['league_name'] == league].groupby(['club_name','isOver25']).count()['short_name'].reset_index()
plot_df = pd.pivot_table(plot_df, values='short_name', index=['club_name'],
                columns=['isOver25'], aggfunc=np.sum).reset_index()
x = plot_df['club_name']
y = plot_df.iloc[:,-2:].div(plot_df.iloc[:,-2:].sum(axis=1), axis=0)
fig2 = go.Figure()
fig2.add_trace(go.Bar(y=x,x=y[True],name='Over 25',orientation='h', marker=dict(color='#5000bf',line=dict(color='DarkSlateGrey', width=1.1))))
fig2.add_trace(go.Bar(y=x,x=y[False],name='Under 25',orientation='h', marker=dict(color='rgb(255,171,0)',line=dict(color='DarkSlateGrey', width=1.1))))

fig2.update_yaxes(tickfont = dict(size=10))
fig2.update_layout(barmode='stack', 
                    title = dict(text ='Players over 25 and under 25 years old by club',
                                    font = dict(size =14)),
                    plot_bgcolor = 'rgba(0, 0, 0, 0)',
                    paper_bgcolor = 'rgba(0, 0, 0, 0)',    
                    legend=dict(title = dict(text ="Age of players"),
                                    orientation="h",
                                    yanchor="bottom",
                                    y=1,
                                    xanchor="right",
                                    x=0.95),
                    margin=dict(r=30, t=100, b=70),)
return fig1,fig2

Top 100 players with highest overall rating in Premier League